### Testing vsearch mergepairs options

In [1]:
import ipyrad as ip
print ip.__version__

0.1.19


In [3]:
data = ip.Assembly("mergetest")

  New Assembly: mergetest


In [4]:
data.get_params()

  1   working_directory           .                                            
  2   raw_fastq_path              ./*.fastq                                    
  3   barcodes_path               ./*.barcodes.txt                             
  4   sorted_fastq_path                                                        
  5   assembly_method             denovo                                       
  6   reference_sequence                                                       
  7   datatype                    rad                                          
  8   restriction_overhang        ('TGCAG', '')                                
  9   max_low_qual_bases          5                                            
  10  phred_Qscore_offset         33                                           
  11  mindepth_statistical        6                                            
  12  mindepth_majrule            6                                            
  13  maxdepth                    1000  

In [9]:
data.dirs.edits = "mergetest"
import os
os.mkdir(data.dirs.edits)

In [10]:
sample = ip.core.sample.Sample('merge')

In [17]:
sample.files.edits = ("merge_R1_.fastq", "merge_R2_.fastq")
sample.files.edits = [("merge_R1_.fastq", "merge_revcomp_R2_.fastq")]


In [27]:
import os
import subprocess

def merge_fastq_pairs(data, sample):
    """ Merge paired fastq reads. """
    cmd = data.bins.vsearch \
         +" --fastq_mergepairs "+sample.files.edits[0][0] \
         +" --reverse "+sample.files.edits[0][1] \
         +" --fastaout "+os.path.join(data.dirs.edits, sample.name+".merge") \
         +" --fastaout_notmerged_fwd "+os.path.join(data.dirs.edits, sample.name+".R1.notmerge") \
         +" --fastaout_notmerged_rev "+os.path.join(data.dirs.edits, sample.name+".R2.notmerge") \
         +" --fastq_allowmergestagger" \
         +" --fastq_maxdiffs "+str(4) \
         +" --fastq_maxns "+str(data.paramsdict["max_low_qual_bases"]) \
         +" --fastq_minlen "+str(32) 
    print cmd
    subprocess.check_output(cmd, shell=True)


In [28]:
ls

data/                    mergetest/            testrad/
data1_results.tex        numba_test.ipynb      test_rad/
dstats.ipynb             params.txt            test_rad.ipynb
ipyrad_test.assembly     quickguide_API.ipynb  test_refseq/
ipytesting.ipynb         test_clust_0.85/      test_refseq.ipynb
mergepairs.ipynb         test_pairddrad/       Untitled1.ipynb
merge_R1_.fastq          test_pairddrad.ipynb  Untitled.ipynb
merge_R2_.fastq          test_pairgbs/         untitled.txt
merge_revcomp_R2_.fastq  test_pairgbs.ipynb


In [29]:
merge_fastq_pairs(data, sample)

/home/deren/Documents/ipyrad/bin/vsearch-linux-x86_64 --fastq_mergepairs merge_R1_.fastq --reverse merge_revcomp_R2_.fastq --fastaout mergetest/merge.merge --fastaout_notmerged_fwd mergetest/merge.R1.notmerge --fastaout_notmerged_rev mergetest/merge.R2.notmerge --fastq_allowmergestagger --fastq_maxdiffs 4 --fastq_maxns 5 --fastq_minlen 32


CalledProcessError: Command '/home/deren/Documents/ipyrad/bin/vsearch-linux-x86_64 --fastq_mergepairs merge_R1_.fastq --reverse merge_revcomp_R2_.fastq --fastaout mergetest/merge.merge --fastaout_notmerged_fwd mergetest/merge.R1.notmerge --fastaout_notmerged_rev mergetest/merge.R2.notmerge --fastq_allowmergestagger --fastq_maxdiffs 4 --fastq_maxns 5 --fastq_minlen 32' returned non-zero exit status 1

In [1]:
import ipyrad as ip
data = ip.load.load_assembly("test_rad/data1.assembly")
samples = data.samples
chunk = "test_rad/data1_c

In [ ]:
def muscle_align_across(args):
    """ aligns reads, does split then aligning for paired reads """
    ## parse args
    data, samples, chunk = args
    snames = [sample.name for sample in samples]

    ## data are already chunked, read in the whole thing
    infile = open(chunk, 'rb')
    clusts = infile.read().split("//\n//\n")[:-1]
    out = []

    ## array to store indel information (not being used yet!)
    maxlen = data._hackersonly["max_fragment_length"]
    if 'pair' in data.paramsdict["datatype"]:
        maxlen *= 2
    indels = np.zeros((len(clusts), len(samples), maxlen), dtype=np.int8)

    ## iterate over clusters and align
    loc = 0
    for loc, clust in enumerate(clusts):
        stack = []
        lines = clust.strip().split("\n")
        names = [i.split()[0][1:] for i in lines]
        seqs = [i.split()[1] for i in lines]

        ## append counter to end of names b/c muscle doesn't retain order
        names = [j+";*"+str(i) for i, j in enumerate(names)]

        ## don't bother aligning singletons
        if len(names) <= 1:
            if names:
                stack = [names[0]+"\n"+seqs[0]]
        else:
            ## split seqs before align if PE. If 'ssss' not found (single end 
            ## or merged reads) then `except` will pass it to SE alignment. 
            try:
                seqs1 = [i.split("nnnn")[0] for i in seqs] 
                seqs2 = [i.split("nnnn")[1] for i in seqs]

                string1 = muscle_call(data, names[:200], seqs1[:200])
                string2 = muscle_call(data, names[:200], seqs2[:200])
                anames, aseqs1 = parsemuscle(string1)
                anames, aseqs2 = parsemuscle(string2)

                ## resort so they're in same order
                aseqs = [i+"nnnn"+j for i, j in zip(aseqs1, aseqs2)]
                somedic = OrderedDict()
                for i in range(len(anames)):
                    somedic[anames[i].rsplit(';', 1)[0]] = aseqs[i]
                    locinds = np.array(aseqs[i] == "-", dtype=np.int32)
                    ## store the indels
                    indels[loc][snames.index(anames[i]\
                                .rsplit("_", 1)[0])][:maxlen] = locinds
            except IndexError:
                string1 = muscle_call(data, names[:200], seqs[:200])
                anames, aseqs = parsemuscle(string1)
                somedic = OrderedDict()
                for i in range(len(anames)):
                    somedic[anames[i].rsplit(";", 1)[0]] = aseqs[i]
                    locinds = np.array(aseqs[i] == "-", dtype=np.int32)
                    ## store the indels
                    indels[loc][snames.index(anames[i]\
                                .rsplit("_", 1)[0])][:maxlen] = locinds

            for key in somedic.iterkeys():
                stack.append(key+"\n"+somedic[key])

        if stack:
            out.append("\n".join(stack))

    ## write to file after
    infile.close()
    outfile = open(chunk, 'wb')#+"_tmpout_"+str(num))
    outfile.write("\n//\n//\n".join(out)+"\n")#//\n//\n")
    outfile.close()

    return chunk, indels, loc+1

In [2]:
import numpy as np
import h5py
import ipyrad as ip

DEBUG:ipyrad:H4CKERZ-mode: __loglevel__ = DEBUG


In [7]:
data1 = ip.load.load_assembly("test_rad/data1.assembly")
sample = data1.samples["1A_0"]

  loading Assembly: data1 [test_rad/data1.assembly]


In [15]:
old_h5 = h5py.File(sample.files.database, 'r')
old_h5.keys()
catarr = old_h5["catg"]

In [17]:
catarr[0]

array([[ 0,  0, 19,  0],
       [ 0,  0,  0, 19],
       [19,  0,  0,  0],
       [ 0, 19,  0,  0],
       [ 0,  0,  0, 19],
       [ 0,  0,  0, 19],
       [19,  0,  0,  0],
       [ 0,  0,  0, 19],
       [ 0, 19,  0,  0],
       [ 0,  0,  0, 19],
       [ 0,  0, 19,  0],
       [19,  0,  0,  0],
       [ 0,  0, 19,  0],
       [ 0, 19,  0,  0],
       [ 0,  0, 19,  0],
       [19,  0,  0,  0],
       [ 0,  0, 19,  0],
       [ 0, 19,  0,  0],
       [ 0,  0,  0, 19],
       [ 0,  0, 19,  0],
       [ 0,  0,  0, 19],
       [ 0,  0,  0, 19],
       [19,  0,  0,  0],
       [19,  0,  0,  0],
       [ 0, 19,  0,  0],
       [ 0,  0, 19,  0],
       [ 0,  0,  0, 19],
       [ 0, 19,  0,  0],
       [ 0,  0, 19,  0],
       [ 0,  0, 19,  0],
       [19,  0,  0,  0],
       [ 0,  0, 19,  0],
       [19,  0,  0,  0],
       [ 0,  0,  0, 19],
       [ 0,  0, 19,  0],
       [ 0, 19,  0,  0],
       [ 0,  0, 19,  0],
       [ 0,  0, 19,  0],
       [ 0,  0,  0, 19],
       [ 0,  0,  0, 19],


### individual catg array

In [28]:
io_one = "/home/deren/Documents/ipyrad/tests/test_pairgbs/test_pairgbs_consens/1A0.hdf5"
one = h5py.File(io_one, 'r')
print one.keys()

[u'1A0']


In [9]:
io_one = "/home/deren/Documents/ipyrad/tests/test_rad/data1_consens/1A_0.hdf5"
#io_one = "/home/deren/Documents/ipyrad/tests/test_rad/data1_consens/1A_0.catg"

one = h5py.File(io_one, 'r')
print one.keys()

[u'1A_0']


In [5]:
dat = one["catg"]
print dat[0][:10]
one.close()

[[ 0  0 19  0]
 [ 0  0  0 19]
 [19  0  0  0]
 [ 0 19  0  0]
 [ 0  0  0 19]
 [ 0  0  0 19]
 [19  0  0  0]
 [ 0  0  0 19]
 [ 0 19  0  0]
 [ 0  0  0 19]]


In [10]:
dat = one["1A_0"]
print dat[0][:10]
one.close()

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [31]:
## 
h5 = "/home/deren/Documents/ipyrad/tests/test_pairgbs/test_pairgbs_consens/test_pairgbs.hdf5"
io5 = h5py.File(h5, 'r')
print io5.keys()
print io5

[u'catgs', u'edges', u'filters', u'seqs']
<HDF5 file "test_pairgbs.hdf5" (mode r)>


In [2]:
## 
h5 = "/home/deren/Documents/ipyrad/tests/test_rad/data1_consens/data1.hdf5"
io5 = h5py.File(h5, 'r')
print io5.keys()
print io5

[u'catgs', u'edges', u'filters', u'seqs']
<HDF5 file "data1.hdf5" (mode r)>


In [3]:
io5['catgs'][0, :, :10]

array([[[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
      

In [40]:
singlecat = "/home/deren/Documents/ipyrad/tests/test_pairgbs/test_pairgbs_consens/1A0.catg"
io5 = h5py.File(singlecat, 'r')


In [44]:
catgs = io5['catg']
print catgs.shape
print catgs[0:3, :10, :]

(200, 300, 4)
[[[ 0  0 20  0]
  [ 0  0  0 20]
  [20  0  0  0]
  [ 0 20  0  0]
  [ 0  0  0 20]
  [19  0  0  1]
  [20  0  0  0]
  [ 0  0 20  0]
  [ 0  0  0 20]
  [ 0  0  0 20]]

 [[ 0  0 20  0]
  [ 0  0  0 20]
  [20  0  0  0]
  [ 0 20  0  0]
  [ 0  0  0 20]
  [20  0  0  0]
  [ 0  0 20  0]
  [20  0  0  0]
  [ 0  0  0 20]
  [20  0  0  0]]

 [[ 0  0 20  0]
  [ 0  0  0 20]
  [20  0  0  0]
  [ 0 20  0  0]
  [ 0  0  0 20]
  [ 0  0 20  0]
  [ 0  0  0 20]
  [ 0  0  0 20]
  [ 0  0 20  0]
  [ 0  0  0 20]]]


In [46]:
io5.close()

In [45]:
filt = io5['filters']
filt[:100]

KeyError: "Unable to open object (Object 'filters' doesn't exist)"

In [5]:
seqs = io5["seqs"]
seqs[0]

array(['', '', '', '', '', '', '', '', '', '', '', ''], 
      dtype='|S300')

In [6]:
seqs.attrs['samples']

array(['1A0', '1B0', '1C0', '1D0', '2E0', '2F0', '2G0', '2H0', '3I0',
       '3J0', '3K0', '3L0'], 
      dtype='|S50')

In [10]:
np.array(["R1_L", "R1_R", "R2_L", "R2_R"])

array(['R1_L', 'R1_R', 'R2_L', 'R2_R'], 
      dtype='|S4')

In [12]:
a = np.array([0, 1, 0, 1])
np.array(a==1, dtype=np.bool)

array([False,  True, False,  True], dtype=bool)

In [21]:
aa = np.where(a==1)
bb = np.where(a==0)
inds = np.concatenate([aa[0],bb[0]])
a[inds]

array([1, 1, 0, 0])

In [27]:
aseqs = ["AAAAAAAAAAAAAnnnnBBBBBBBBBBB",
         "AAAAAAAAAAAAAnnnnCCCCCCCCCCC"]

In [36]:
np.where(list(aseqs[0])=='n')

(array([], dtype=int64),)

In [41]:
np.array(list(aseqs[0]))=='n'

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False, False], dtype=bool)